In [13]:
from dotenv import load_dotenv
load_dotenv()

from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [46]:
# initialize local database
engine = create_engine("sqlite:///../sql/Chinook.db")
database = SQLDatabase(
    engine, # connection to the database
    sample_rows_in_table_info=3 # number of rows to show in table info
)

## Prompt Template

In [47]:
prompt_text_to_sql = """
You are a SQL Expert
Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Plase following this rules:
1. If the user requests all the data or uses a DISTINCT clause, then there is no need to use LIMIT. However, If the user does not specify a limit in the query, then the query uses a LIMIT clause of at most 10.
2. Pay attention to not run the SQL query if the question involves create table or drop table and instead answer the question with "SELECT 'Pembuatan atau Penghapusan Tabel dilarang' AS result".

Based on the table schema below, write a SQL query that would answer the user's question:

{table_info}

Question: {input}

SQL Query:
"""

prompt_template_text_to_sql = PromptTemplate(
    template=prompt_text_to_sql,
    input_variables=["table_info", "input", "dialect"], # variable name from the prompt
)

In [18]:
prompt_sql_to_answer = """
Please make sure you complete the objective above with the following rules:
1. You shall provide response in the same language as the user language. If  the user asks in Indonesian, you shall response in Indonesian
2. Answer straight away. Dont add things like "Jawaban:", "Answer:", "Final Answer:" and such.
3. If you want to display data, then display everything again in the answer.
4. Based on the current conversation, question, sql query, and sql response, please write a natural language response.
5. None or empty in the SQL Response means the data is not available.

Question: {input}
SQL Query: {query}
SQL Response: {response}
"""

prompt_template_sql_to_answer = PromptTemplate(
    template=prompt_sql_to_answer,
    input_variables=["input", "query", "response"],
)

In [21]:
# define to get dialect
def get_dialect(_):
    return database.dialect

print("Dialect:", get_dialect({}))

Dialect: sqlite


In [22]:
# define to get schema from table
def get_schema(_):
    return database.get_table_info()

print("Schema:", get_schema({}))

Schema: 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENC

In [23]:
# define to get response from query sql
def get_response(query):
    response = database.run(query)
    return response

print("Response:", get_response("SELECT * FROM Album LIMIT 5"))

Response: [(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]


## Create Chain

In [24]:
text2sql_model = ChatOpenAI(model="gpt-4o", temperature=0.0)
sql2text_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

/tmp/ipykernel_1075926/955789175.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  text2sql_model = ChatOpenAI(model="gpt-4o", temperature=0.0)


In [25]:
# Create chain text to sql using LangChain Expression Language (LCEL)
sql_chain_response = (
    RunnablePassthrough.assign(
        table_info=get_schema,
        dialect=get_dialect,
    )
    | prompt_template_text_to_sql
    | text2sql_model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# Create chain sql to natural language
natural_chain_response = (
    RunnablePassthrough.assign(
        response=lambda x: get_response(x["query"]),
        query=lambda x: x["query"]
    )
    | natural_prompt
    | chat_model
    | StrOutputParser()
)

In [29]:
sql = sql_chain_response.invoke({"input": "berapa total customer ?"})
print("Query: ",sql)

Query:  ```sql
SELECT COUNT("CustomerId") AS "TotalCustomers" FROM "Customer";
```


In [37]:
# Postprocessing query sql
def postprocessing_query_sql(sql):
    sql = re.sub(r"^\`{3}sql\n(.*?)\`{3}$", r"\1", sql.strip(), flags=re.DOTALL)
    return sql

print("After Postprocessing: ", postprocessing_query_sql(sql))

After Postprocessing:  SELECT COUNT("CustomerId") AS "TotalCustomers" FROM "Customer";


In [36]:
get_response(sql)

'[(59,)]'

In [38]:
# Create chain sql to natural language
natural_chain_response = (
    RunnablePassthrough.assign(
        response=lambda x: get_response(postprocessing_query_sql(x["query"])),
        query=lambda x: x["query"]
    )
    | prompt_template_sql_to_answer
    | sql2text_model
    | StrOutputParser()
)

In [39]:
natural_chain_response.invoke(
    {"input": "berapa total customer ?", "query": sql}
)

'Total customer yang terdaftar adalah 59.'

In [40]:
def qna(question):
    sql_query = sql_chain_response.invoke({"input": question})
    natural_response = natural_chain_response.invoke(
        {"input": question, "query": sql_query}
    )
    return natural_response

In [43]:
answer = qna("Berikan 5 album teratas")

In [45]:
print(answer)

Berikut adalah 5 album teratas:

1. For Those About To Rock We Salute You - Artis ID: 1
2. Balls to the Wall - Artis ID: 2
3. Restless and Wild - Artis ID: 2
4. Let There Be Rock - Artis ID: 1
5. Big Ones - Artis ID: 3
